<a href="https://colab.research.google.com/github/jcdevaney/metricData/blob/main/metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Libraries and Data

In [2]:
!pip install libfmp
!pip install mir_eval
import numpy as np, os, scipy
import pandas as pd
import mir_eval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.7/90.7 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for mir_eval: filename=mir_eval-0.7-py3-none-any.whl size=100703 sha256=16b1020f35f61dd6c61254de9e83480d9612f45cc12893b615e4f68c1ed1800b
  Stored in directory: /root/.cache/pip/wheels/3e/2f/0d/dda9c4c77a170e21356b6afa2f7d9bb078338634ba05d94e3f
Successfully built mir_eval


# Define Metrics

In [ ]:
from libfmp.c3 import normalize_feature_sequence

def cossimBM(vec1, vec2, threshold=0.001):
    vec1_norm = normalize_feature_sequence(vec1, norm='2', threshold=threshold)
    vec2_norm = normalize_feature_sequence(vec2, norm='2', threshold=threshold)
    cosine_sim = np.sum(np.multiply(vec1_norm, vec2_norm))/vec2_norm.shape[1]
    return cosine_sim

In [ ]:
# symmetrized version of above measure

# def predictionsInsertionsSymmetric(label1BM, label2BM):

#     label1=set(label1BM.nonzero()[0])
#     label2=set(label2BM.nonzero()[0])

#     if (len(label1)==0) & (len(label2)==0):
#         A = 1
#     elif len(label1)==0:
#         A = predictionsInsertionsBM(label1BM, label2BM)
#     elif len(label2)==0:
#         A = predictionsInsertionsBM(label2BM, label1BM)
#     else:
#         A = (predictionsInsertionsBM(label1BM, label2BM) + predictionsInsertionsBM(label2BM, label1BM))/2

#     return A

def predictionsInsertionsBM(estimateBM, referenceBM):
    estimate=set(estimateBM.nonzero()[0])
    reference=set(referenceBM.nonzero()[0])
    le = len(estimate)
    lr = len(reference)

    # C is the number of predicted notes in the estimate that occur in the reference (ground truth)
    C = len(estimate.intersection(reference))

    # I is the number of insertions (extra predicted notes) in the estimate that are not present in
    # the reference (ground truth)
    I = max(len(estimate.difference(reference)), len(reference.difference(estimate)))

    maxlength = max(le, lr)

    if maxlength==0:
        A = 1

    else:
        # accuracy measurement for each chord estimate, scaled between 0 and 1
        A = (C - I + maxlength) / (2*maxlength)
        # A = (C - I + len(reference)) / (2*len(reference))

    return A

# Example Use of Metric Functions

In [ ]:
# Labels
# estimates
dminlab = 'D:min'

# reference
FMajlab = 'F:maj'

rootnum1, bitmap1, bassnum1 = mir_eval.chord.encode(dminlab, reduce_extended_chords=False, strict_bass_intervals=False)
absolute_pcs1 = mir_eval.chord.rotate_bitmap_to_root(bitmap1, rootnum1)
rootnum2, bitmap2, bassnum2 = mir_eval.chord.encode(label2, reduce_extended_chords=False, strict_bass_intervals=False)
absolute_pcs2 = mir_eval.chord.rotate_bitmap_to_root(FMajlab, rootnum2)

cossimBMres = cossimBM(np.expand_dims(absolute_pcs1, axis=-1), np.expand_dims(absolute_pcs2, axis=-1))
predictionsInsertionsBMres = predictionsInsertionsBM(np.expand_dims(absolute_pcs1, axis=-1), np.expand_dims(absolute_pcs2, axis=-1))